In [ ]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline

Multiclass margin classifier {#multiclass_margin_classifier}
============================

::: {.meta}
:property=\"og:description\": Using PyTorch to implement a multiclass
quantum variational classifier on MNIST data. :property=\"og:image\":
<https://pennylane.ai/qml/_images/margin_2.png>
:::

::: {.related}
tutorial\_variational\_classifier Variational classifier
tutorial\_data\_reuploading\_classifier Data-reuploading classifier
:::

*Author: Safwan Hossein --- Posted: 09 April 2020. Last updated: 28
January 2021.*

In this tutorial, we show how to use the PyTorch interface for PennyLane
to implement a multiclass variational classifier. We consider the iris
database from UCI, which has 4 features and 3 classes. We use multiple
one-vs-all classifiers with a margin loss (see [Multiclass Linear
SVM](http://cs231n.github.io/linear-classify/)) to classify data. Each
classifier is implemented on an individual variational circuit, whose
architecture is inspired by [Farhi and Neven
(2018)](https://arxiv.org/abs/1802.06002) as well as [Schuld et al.
(2018)](https://arxiv.org/abs/1804.00633).

| 

![](../demonstrations/multiclass_classification/margin_2.png){.align-center
width="50.0%"}

| 

Initial Setup
-------------

We import PennyLane, the PennyLane-provided version of NumPy, relevant
torch modules, and define the constants that will be used in this
tutorial.

Our feature size is 4, and we will use amplitude embedding. This means
that each possible amplitude (in the computational basis) will
correspond to a single feature. With 2 qubits (wires), there are 4
possible states, and as such, we can encode a feature vector of size 4.


In [ ]:
# the number of the required qubits is calculated from the number of features

Quantum Circuit
===============

We first create the layer that will be repeated in our variational
quantum circuits. It consists of rotation gates for each qubit, followed
by entangling/CNOT gates


We now define the quantum nodes that will be used. As we are
implementing our multiclass classifier as multiple one-vs-all
classifiers, we will use 3 QNodes, each representing one such
classifier. That is, `circuit1` classifies if a sample belongs to class
1 or not, and so on. The circuit architecture for all nodes are the
same. We use the PyTorch interface for the QNodes. Data is embedded in
each circuit using amplitude embedding.

::: {.note}
::: {.title}
Note
:::

For demonstration purposes we are using a very simple circuit here. You
may find that other choices, for example more elaborate measurements,
increase the power of the classifier.
:::


The variational quantum circuit is parametrized by the weights. We use a
classical bias term that is applied after processing the quantum
circuit\'s output. Both variational circuit weights and classical bias
term are optimized.


Loss Function
=============

Implementing multiclass classifiers as a number of one-vs-all
classifiers generally evokes using the margin loss. The output of the
$i$ th classifier, $c_i$ on input $x$ is interpreted as a score, $s_i$
between \[-1,1\]. More concretely, we have:

$$s_i = c_i(x; \theta)$$

The multiclass margin loss attempts to ensure that the score for the
correct class is higher than that of incorrect classes by some margin.
For a sample $(x,y)$ where $y$ denotes the class label, we can
analytically express the mutliclass loss on this sample as:

$$L(x,y) = \sum_{j \ne y}{\max{\left(0, s_j - s_y + \Delta)\right)}}$$

where $\Delta$ denotes the margin. The margin parameter is chosen as a
hyperparameter. For more information, see [Multiclass Linear
SVM](http://cs231n.github.io/linear-classify/).


Classification Function
=======================

Next, we use the learned models to classify our samples. For a given
sample, compute the score given to it by classifier $i$, which
quantifies how likely it is that this sample belongs to class $i$. For
each sample, return the class with the highest score.


Data Loading and Processing
===========================

Now we load in the iris dataset and normalize the features so that the
sum of the feature elements squared is 1 ($\ell_2$ norm is 1).


In [ ]:
# Create a train and test split.

Training Procedure
==================

In the training procedure, we begin by first initializing randomly the
parameters we wish to learn (variational circuit weights and classical
bias). As these are the variables we wish to optimize, we set the
`requires_grad` flag to `True`. We use minibatch training---the average
loss for a batch of samples is computed, and the optimization step is
based on this. Total training time with the default parameters is
roughly 15 minutes.


In [ ]:
# We now run our training algorithm and plot the results. Note that
# for plotting, the matplotlib library is required